<a href="https://colab.research.google.com/github/hongkwangpyo/2020_deep_learning/blob/master/0811_6_keraas_RNN_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import random
import re
from sklearn.preprocessing import *

In [2]:
from google.colab import files
uploaded = files.upload()

Saving lastofus.txt to lastofus.txt


In [ ]:
file = 'lastofus.txt'
with open(file, encoding='cp949') as f:
  line = f.readlines()
  for i in line:
    print(i, end='//')

In [4]:
text = ''
array = []
with open(file, encoding='cp949') as f:
  line = f.readlines()
  text = text.join([i for i in line if re.match(r'^[A-Z].*:', i)])

  for t in text.split('\n')[:1000]:
    array.append(t)
array[:5]

["Joel: Tommy, I-...Tommy. Tommy, listen to me. He's the contractor, okay? I",
 'Sarah: Hey.',
 'Joel: Scoot.',
 'Sarah: Fun day at work, huh?',
 "Joel: What are you still doing up? It's late."]

In [23]:
token = Tokenizer(lower = False, filters='.,?;\'"-')
token.fit_on_texts(array)

In [24]:
token.word_index['hey']

318

In [25]:
seq = token.texts_to_sequences(array)
seq[0]

[1, 130, 3, 130, 130, 626, 8, 28, 103, 4, 5, 627, 104, 3]

In [26]:
seq = pad_sequences(seq, maxlen=10)
seq[0]

array([130, 626,   8,  28, 103,   4,   5, 627, 104,   3], dtype=int32)

In [27]:
seq.shape

(1000, 10)

In [28]:
x = seq

In [29]:
x

array([[ 130,  626,    8, ...,  627,  104,    3],
       [   0,    0,    0, ...,    0,   17,   58],
       [   0,    0,    0, ...,    0,    1,  628],
       ...,
       [  57, 1382,   22, ...,   21,   55,  418],
       [   0,    0,    0, ...,    0,    7,  328],
       [   0,    0,    0, ...,    1, 1384,    9]], dtype=int32)

In [30]:
y = np.vstack((x[1:], x[0]))
y 

array([[   0,    0,    0, ...,    0,   17,   58],
       [   0,    0,    0, ...,    0,    1,  628],
       [   0,    0,    0, ...,  111,  266,  331],
       ...,
       [   0,    0,    0, ...,    0,    7,  328],
       [   0,    0,    0, ...,    1, 1384,    9],
       [ 130,  626,    8, ...,  627,  104,    3]], dtype=int32)

In [31]:
x = x.reshape(-1, 10, 1)
y = y.reshape(-1, 10, 1)

In [32]:
from keras.utils import to_categorical
y = to_categorical(y)

In [33]:
y.shape

(1000, 10, 1385)

In [34]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(128, return_sequences=True), input_shape=(10, 1))) 
model.add(Bidirectional(SimpleRNN(128, return_sequences=True))) 
model.add(Dense(1385, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 10, 256)           33280     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 10, 256)           98560     
_________________________________________________________________
dense_3 (Dense)              (None, 10, 1385)          355945    
Total params: 487,785
Trainable params: 487,785
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(x, y, epochs=10, batch_size=1)

Epoch 1/10
1000/1000 [==============================] - 17s 17ms/step - loss: 5.0162 - accuracy: 0.3205
Epoch 2/10
1000/1000 [==============================] - 17s 17ms/step - loss: 4.6824 - accuracy: 0.3289
Epoch 3/10
1000/1000 [==============================] - 17s 17ms/step - loss: 4.5032 - accuracy: 0.3286
Epoch 4/10
1000/1000 [==============================] - 17s 17ms/step - loss: 4.3558 - accuracy: 0.3261
Epoch 5/10
1000/1000 [==============================] - 17s 17ms/step - loss: 4.1967 - accuracy: 0.3278
Epoch 6/10
1000/1000 [==============================] - 17s 17ms/step - loss: 4.0502 - accuracy: 0.3247
Epoch 7/10
1000/1000 [==============================] - 17s 17ms/step - loss: 3.9070 - accuracy: 0.3242
Epoch 8/10
1000/1000 [==============================] - 17s 17ms/step - loss: 3.7665 - accuracy: 0.3207
Epoch 9/10
1000/1000 [==============================] - 16s 16ms/step - loss: 3.6293 - accuracy: 0.3239
Epoch 10/10
1000/1000 [==============================] - 16s 16m

In [55]:
model.save('lastofus')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: lastofus/assets


In [56]:
model1 = load_model('lastofus')

In [58]:
pred = model.predict(x[7:8])
pred = np.argmax(pred, axis=2)

In [59]:
#token.word_index

In [60]:
idx_word = {}
for w in token.word_index:
  idx_word[token.word_index[w]] = w

In [62]:
ori_text = x[7:8].flatten()

temp = ''

for w in ori_text:
  if w != 0:
    temp += idx_word[w]
    temp += ' '

In [63]:
for i in range(10):
  temp = ''
  ran = random.randrange(0, len(x))
  pred = model.predict(np.expand_dims(x[ran], axis=0))

  pred = np.argmax(pred, axis=2)

  for i in pred:
    for w in i:
      if w != 0:
        temp += idx_word[w]
        temp += ' '
      # temp += '\n'
    print('-'*30)
    print(array[ran])
    print(temp)

------------------------------
Joel: C'mon, keep up.
Copy for here 
------------------------------
Ellie: What the fuck! I can't believe we did that.
be I than here 
------------------------------
Tess: Oh, shit. Look.
Trust Okay 
------------------------------
Tommy: C'mon!
figured Trust him 
------------------------------
Ellie: Holy moley. I guess this is what these buildings look like up close.
hadn would s here that 
------------------------------
Joel: Jesus Christ, how did this happen?
hour Quit s you that 
------------------------------
Sarah: There you are.
m okay 
------------------------------
Sarah: How do you know?
Citizens required carry We re blind 
------------------------------
Ellie: Okay... Alright, that should do it.
I re you Pssh 
------------------------------
Tess: I'm fine.
morning Want him 
